In [1]:
import os

os.environ['HTTP_PROXY'] = "127.0.0.1:33210"
os.environ['HTTPS_PROXY'] = "127.0.0.1:33210"
os.environ["OPENAI_API_KEY"] = "you-api_key"

In [2]:
import jieba

# 利用jieba_fast进行分词
def jieba_extract_keywords(question):
    # 使用jieba进行分词
    keywords = jieba.cut(question)
    return keywords


# 利用chatgpt进行分词
def gpt_extract_keywords(question):
    from langchain.prompts import ChatPromptTemplate
    prompts = f"""
        ----
        {question}
        ----
        对以上文本进行关键词提取，要求一个关键词占一行，只需要返回关键词
        """
    promptsTemplate = ChatPromptTemplate.from_template(prompts)
    from langchain.chains import LLMChain
    from langchain.chat_models import ChatOpenAI
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0)
    chain = LLMChain(llm=llm, prompt=promptsTemplate)
    response = chain.run(question=question)
    return response


In [3]:
import PyPDF2


def search_keywords_in_pdf(file_path, keywords):
    keyword_count_per_page = {}

    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()

            # 计算当前页面的关键词出现次数
            count = sum(text.count(keyword) for keyword in keywords)

            # 如果关键词出现次数大于0，则记录该页码和次数
            if count > 0:
                keyword_count_per_page[page_num + 1] = count

    # 根据关键词出现次数从大到小排序页面号
    sorted_page_numbers = [k for k, v in sorted(keyword_count_per_page.items(), key=lambda item: item[1], reverse=True)]

    return sorted_page_numbers


def get_page_content(file_path, page_number):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # 确保页面号在有效范围内
        if page_number - 1 < len(reader.pages):
            page = reader.pages[page_number - 1]
            text = page.extract_text()
            return text
        else:
            return "指定的页码超出了PDF文件的范围"


In [4]:
# 用户输入的问题
question = "电子齿轮比参数设定"

# 从问题中提取关键词
keywords = gpt_extract_keywords(question)
print(keywords)

import ast

keywords = keywords.splitlines()

print(keywords)
print(type(keywords))

# # 指定PDF文件路径
file_path = 'KT820说明书V2.0 201909.pdf'

# 搜索关键词，并打印匹配的页码
pages = search_keywords_in_pdf(file_path, keywords)
print(f"关键词匹配的页面有: {pages}")

AuthenticationError: Incorrect API key provided: you-api_key. You can find your API key at https://platform.openai.com/account/api-keys.

In [ ]:
os.environ["OPENAI_API_KEY"] = "you-api_key"
from langchain.chat_models import ChatOpenAI

from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("KT820说明书V2.0 201909.pdf")
unprocessed_documents = loader.load_and_split()

print(unprocessed_documents[1].metadata['page'])
documents = []
for page in pages:
    for i in range(len(unprocessed_documents)):
        if unprocessed_documents[i].metadata['page'] == page - 1:
            documents.append(unprocessed_documents[i])

In [ ]:

print(documents)

In [ ]:

from langchain.text_splitter import CharacterTextSplitter

os.environ["OPENAI_API_KEY"] = "you-api_key"
documents = documents[:20]      # 只提取最符合的前二十项
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613"), chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

prompt = f"""
    下面是用户提的问题,以'''为分割符号
    '''
    {question}
    '''
"""

res = qa(prompt)
answer, docs = res['result'], res['source_documents']
print(answer)